In [12]:
import numpy as np
import pandas as pd 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial import distance_matrix
from scipy.spatial.distance import squareform
from sklearn.preprocessing import MinMaxScaler
from string import punctuation
import re


In [2]:
data = pd.read_csv("wine400_fin.csv")

# 확인
data.head()

,Unnamed: 0,주종,중분류,상품명,생산국가,생산자,주요품종,도수,음용온도,추천음식
0,0,wine,레드,"GCF, 꼬뜨 드 뵈프",프랑스,GCF 그룹 GCF Group,"시라/쉬라즈 (Syrah/Shiraz) , 마르셀란 (Marselan)",13~14 %,16~18 ℃,"뵈프 부르기뇽, 안심 찹 스테이크, 로스트 비프 스테이크 등과 잘 어울린다."
1,1,wine,화이트,"가트, 리슬링",호주,가트 와인즈 Gatt Wines,리슬링 (Riesling) 100%,NaN,10~12 ℃,"샐러드, 해산물, 치즈 등과 잘 어울린다."
2,2,wine,레드,갈로 리빙스톤 콩코드,미국,갤로 패밀리 빈야드 Gallo Family Vineyard - E&J Gallo W...,콩코드 (Concord),NaN,14~16 ℃,"소시지, 육포, 떡볶이, 순대, 튀김, 족발, 만두, 자장면 등과 잘 어울린다."
3,3,wine,주정강화,글로리아 토니 포트 와인,포르투갈,빈센테 파리아 비뉴스 Vincente Faria VInhos,"틴타 로리즈 (Tinta Roriz) , 투리가 프란카 (Touriga Franca...",20~21 %,14~16 ℃,"리코타, 크림치즈, 말린과일, 케이크, 초콜렛 등 디저트와 잘 어울린다."
4,4,wine,레드,"기욤 르탕, 루비 루즈",프랑스,기욤 르탕 Guillaume Letang,그르나슈 (Grenache),14~15 %,16~18 ℃,"소고기, 스테이크, 불고기, 양꼬치, 족발, 한국음식, 피자, 햄버거 등과 잘 어울린다."


In [14]:
data['도수'] = [re.sub('[^A-Za-z0-9가-힣]', '', str(title_name)) for title_name in data['도수']]

In [15]:
data

,Unnamed: 0,주종,중분류,상품명,생산국가,생산자,주요품종,도수,음용온도,추천음식
0,0,wine,레드,"GCF, 꼬뜨 드 뵈프",프랑스,GCF 그룹 GCF Group,"시라/쉬라즈 (Syrah/Shiraz) , 마르셀란 (Marselan)",1314,16~18 ℃,"뵈프 부르기뇽, 안심 찹 스테이크, 로스트 비프 스테이크 등과 잘 어울린다."
1,1,wine,화이트,"가트, 리슬링",호주,가트 와인즈 Gatt Wines,리슬링 (Riesling) 100%,nan,10~12 ℃,"샐러드, 해산물, 치즈 등과 잘 어울린다."
2,2,wine,레드,갈로 리빙스톤 콩코드,미국,갤로 패밀리 빈야드 Gallo Family Vineyard - E&J Gallo W...,콩코드 (Concord),nan,14~16 ℃,"소시지, 육포, 떡볶이, 순대, 튀김, 족발, 만두, 자장면 등과 잘 어울린다."
3,3,wine,주정강화,글로리아 토니 포트 와인,포르투갈,빈센테 파리아 비뉴스 Vincente Faria VInhos,"틴타 로리즈 (Tinta Roriz) , 투리가 프란카 (Touriga Franca...",2021,14~16 ℃,"리코타, 크림치즈, 말린과일, 케이크, 초콜렛 등 디저트와 잘 어울린다."
4,4,wine,레드,"기욤 르탕, 루비 루즈",프랑스,기욤 르탕 Guillaume Letang,그르나슈 (Grenache),1415,16~18 ℃,"소고기, 스테이크, 불고기, 양꼬치, 족발, 한국음식, 피자, 햄버거 등과 잘 어울린다."
...,...,...,...,...,...,...,...,...,...,...
394,394,wine,레드,"매티스, 그르나슈",미국,매티스 와이너리 Mathis Winery,"그르나슈 (Grenache) 82%, 까리냥 (Carignan/Carignane) ...",1415,16~18 ℃,"덩어리 째 오븐에 구운 육류와 햄, 스테이크, 라자냐, 치즈 등과 잘 어울린다."
395,395,wine,화이트,"메종 카스텔, 소비뇽 블랑",프랑스,메종 카스텔 Maison Castel,소비뇽 블랑 (Sauvignon Blanc),1314,8~10 ℃,"연어 타르타르 스테이크와 같은 해산물 요리, 샐러드와 같은 야채, 생선 그릴 요리와..."
396,396,wine,스파클링,"메종 카스텔, 퀴베 블랑쉬 브뤼",프랑스,메종 카스텔 Maison Castel,샤르도네 (Chardonnay),1112,6~8 ℃,"디저트, 조개, 생선 요리 또는 반주와 잘 어울린다."
397,397,wine,화이트,"메종 카스텔, 푸이 퓌메",프랑스,메종 카스텔 Maison Castel,소비뇽 블랑 (Sauvignon Blanc),1314,8~10 ℃,"샤프란을 곁들인 생선 스튜 요리 등 생선, 갑각류, 조개 요리, 크림 소스를 곁들인..."


In [16]:
data.to_csv('./wine400_fin.csv', encoding = 'utf-8-sig')

In [25]:
wine = pd.read_csv("wine400_fin.csv")
wine

,Unnamed: 0.1,Unnamed: 0,주종,중분류,상품명,생산국가,생산자,주요품종,도수,음용온도,추천음식
0,0,0,wine,레드,"GCF, 꼬뜨 드 뵈프",프랑스,GCF 그룹 GCF Group,"시라/쉬라즈 (Syrah/Shiraz) , 마르셀란 (Marselan)",13.0,16~18 ℃,"뵈프 부르기뇽, 안심 찹 스테이크, 로스트 비프 스테이크 등과 잘 어울린다."
1,1,1,wine,화이트,"가트, 리슬링",호주,가트 와인즈 Gatt Wines,리슬링 (Riesling) 100%,NaN,10~12 ℃,"샐러드, 해산물, 치즈 등과 잘 어울린다."
2,2,2,wine,레드,갈로 리빙스톤 콩코드,미국,갤로 패밀리 빈야드 Gallo Family Vineyard - E&J Gallo W...,콩코드 (Concord),NaN,14~16 ℃,"소시지, 육포, 떡볶이, 순대, 튀김, 족발, 만두, 자장면 등과 잘 어울린다."
3,3,3,wine,주정강화,글로리아 토니 포트 와인,포르투갈,빈센테 파리아 비뉴스 Vincente Faria VInhos,"틴타 로리즈 (Tinta Roriz) , 투리가 프란카 (Touriga Franca...",20.0,14~16 ℃,"리코타, 크림치즈, 말린과일, 케이크, 초콜렛 등 디저트와 잘 어울린다."
4,4,4,wine,레드,"기욤 르탕, 루비 루즈",프랑스,기욤 르탕 Guillaume Letang,그르나슈 (Grenache),14.0,16~18 ℃,"소고기, 스테이크, 불고기, 양꼬치, 족발, 한국음식, 피자, 햄버거 등과 잘 어울린다."
...,...,...,...,...,...,...,...,...,...,...,...
394,394,394,wine,레드,"매티스, 그르나슈",미국,매티스 와이너리 Mathis Winery,"그르나슈 (Grenache) 82%, 까리냥 (Carignan/Carignane) ...",14.0,16~18 ℃,"덩어리 째 오븐에 구운 육류와 햄, 스테이크, 라자냐, 치즈 등과 잘 어울린다."
395,395,395,wine,화이트,"메종 카스텔, 소비뇽 블랑",프랑스,메종 카스텔 Maison Castel,소비뇽 블랑 (Sauvignon Blanc),13.0,8~10 ℃,"연어 타르타르 스테이크와 같은 해산물 요리, 샐러드와 같은 야채, 생선 그릴 요리와..."
396,396,396,wine,스파클링,"메종 카스텔, 퀴베 블랑쉬 브뤼",프랑스,메종 카스텔 Maison Castel,샤르도네 (Chardonnay),11.0,6~8 ℃,"디저트, 조개, 생선 요리 또는 반주와 잘 어울린다."
397,397,397,wine,화이트,"메종 카스텔, 푸이 퓌메",프랑스,메종 카스텔 Maison Castel,소비뇽 블랑 (Sauvignon Blanc),13.0,8~10 ℃,"샤프란을 곁들인 생선 스튜 요리 등 생선, 갑각류, 조개 요리, 크림 소스를 곁들인..."


In [26]:
wine = wine.drop(['Unnamed: 0.1'],axis=1)

In [27]:
wine = wine.drop(['Unnamed: 0'],axis=1)

In [28]:
wine.to_csv('./wine400_fin.csv', encoding = 'utf-8-sig')